In [1]:
%pip install numpy -q
import numpy as np

Note: you may need to restart the kernel to use updated packages.


In [61]:

A = np.matrix([[3, -4,  1],
               [1,  2,  2],
               [4,  0, -3]
               ])

B = np.matrix([[9], 
               [3],
               [-2]
               ])


In [21]:
print(X)

[[ 3 -4  1]
 [ 1  2  2]
 [ 4  0 -3]]


In [69]:

n = X.shape[0]
a = np.float64(X.copy())
x = np.zeros(n)
c = np.zeros(n)

p = np.zeros(n)
for i in range(n):
    p[i] = 1

for k in range(n - 1):
    
    pv = np.abs(a[k, k])
    r = k
    
    for i in range(k + 1, n):
        pv = np.abs(a[i, k])
        r = i

    if pv == 0: break

    aux = p[k]
    p[k] = p[r]
    p[r] = aux

    for j in range(n):
        aux = a[k, j]
        a[k, j] = a[r, j]
        a[r, j] = aux

    for i in range(k + 1, n):
        m = a[i, k] / a[k, k]
        a[i, k] = m
        for j in range(k + 1, n):
            a[i, j] = a[i, j] - (m * a[k, j])


for i in range(n):
    r = p[i]
    #c[i] = b[r]
    
for i in range(n):
    soma = 0
    for j in range(i - 1):
        soma += (a[i, j] * b[j])
    c[i] -= soma

for i in range(n, 0, -1):
    soma = 0
    for j in range(i + 1, n):
        soma += a[i, j] * x[j]
        x[i] = (y[i] - soma)/a[i, i]

print(x)


[ 0.    -2.625  0.   ]


/tmp/ipykernel_28807/2339217522.py:45: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  c[i] -= soma


In [88]:
import numpy as np

class FatoracaoLU:

    def __init__(self) -> None:
        pass

    @staticmethod
    def _pivot_rows(A: np.matrix) -> tuple[np.ndarray, np.matrix]:
        n = A.shape[0]
        a = np.float64(A.copy())

        # Initialize permutation vector with identity
        p = np.arange(n)  
        
        for k in range(n - 1):

            # Find the pivot
            r = np.argmax(np.abs(a[k:n, k])) + k  
            if np.abs(a[r, k]) == 0:
                return p, A

            # Swap rows in matrix and permutation vector
            if r != k:
                a[[k, r], :] = a[[r, k], :]
                p[[k, r]] = p[[r, k]]

            # Eliminate entries below pivot
            #   Store the multiplier in lower triangular part
            #   Then eliminate
            for i in range(k + 1, n):
                m = a[i, k] / a[k, k]
                a[i, k] = m  
                a[i, k + 1:n] -= m * a[k, k + 1:n]  

        return p, a

    @staticmethod
    def _forward_substitution(p: np.ndarray, a: np.matrix, B: np.ndarray) -> np.ndarray:
        n = a.shape[0]
        y = np.zeros(n)
        b = np.float64(B.copy())

        # Apply permutation to b 
        # and perform forward substitution
        for i in range(n):
            y[i] = b[p[i]].item()
            for j in range(i):
                y[i] -= a[i, j] * y[j]
                
        return y

    @staticmethod
    def _backward_substitution(a: np.matrix, y: np.ndarray) -> np.ndarray:
        n = a.shape[0]
        x = np.zeros(n)

        # Perform backward substitution
        for i in range(n - 1, -1, -1):
            x[i] = y[i]
            for j in range(i + 1, n):
                x[i] -= a[i, j] * x[j]
            x[i] /= a[i, i]

        return x

    @staticmethod
    def solve(A: np.matrix, B: np.ndarray) -> np.ndarray:
        fat = FatoracaoLU()

        p, a = fat._pivot_rows(A)
        y = fat._forward_substitution(p, a, B)
        x = fat._backward_substitution(a, y)

        return x


In [89]:
FatoracaoLU.solve(A, B)

array([ 1., -1.,  2.])

In [71]:

# Assuming X is the matrix and b is the right-hand side vector
n = A.shape[0]
a = np.float64(A.copy())
b = np.float64(B.copy())
x = np.zeros(n)
y = np.zeros(n)
p = np.arange(n)  # To track permutations

# LU Decomposition with partial pivoting
for k in range(n - 1):

    # Find the pivot row
    r = np.argmax(np.abs(a[k:n, k])) + k  # Index of max pivot in column k
    if np.abs(a[r, k]) == 0:
        raise ValueError("Matrix is singular.")

    # Swap rows in a and p
    if r != k:
        a[[k, r], :] = a[[r, k], :]  # Swap rows in matrix
        p[[k, r]] = p[[r, k]]        # Swap corresponding permutation indices

    # Eliminate entries below pivot
    for i in range(k + 1, n):
        m = a[i, k] / a[k, k]
        a[i, k] = m  # Store the multiplier in the lower triangular part
        a[i, k + 1:n] -= m * a[k, k + 1:n]  # Update row i with the elimination

# Forward substitution (Ly = Pb)
for i in range(n):
    y[i] = b[p[i]]  # Apply permutation to b
    for j in range(i):
        y[i] -= a[i, j] * y[j]

# Backward substitution (Ux = y)
for i in range(n - 1, -1, -1):
    x[i] = y[i]
    for j in range(i + 1, n):
        x[i] -= a[i, j] * x[j]
    x[i] /= a[i, i]


print(x)

[ 1. -1.  2.]


/tmp/ipykernel_28807/905324207.py:30: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y[i] = b[p[i]]  # Apply permutation to b


In [57]:
35/8

4.375